# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요 라이브러리 불러오기

In [ ]:
# konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 450kB 35.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from konlpy.tag import Okt, Hannanum, Kkma,Komoran,Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import pyplot as plt
from datetime import datetime as dt
import math

ModuleNotFoundError: ignored

In [ ]:
# matplotlib 한글 폰트 출력코드
# 출처 : 데이터공방( https://kiddwannabe.blog.me)

import matplotlib
from matplotlib import font_manager, rc
import platform

try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass
matplotlib.rcParams['axes.unicode_minus'] = False   

In [ ]:
# 수지님 데이터 불러오는 부분
sj_path = '/content/drive/MyDrive/AniHelp_feat3.csv'
df = pd.read_csv(sj_path, index_col=0)
df.head()

NameError: ignored

In [ ]:
# 정희 데이터 불러오는 부분
jeonghee_path = '/content/drive/MyDrive/TwoPs/AniHelp_feat5.csv'
df = pd.read_csv(jeonghee_path, index_col=0)
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,gender,neutering,털색2,품종2,지역,성격,건강,나이,입양여부,위도,경도
0,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),1,2,흰색,믹스견,광주 남구,0,2,0,0,35.222932,126.881636
1,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,2,흰색,믹스견,광주 남구,0,0,0,1,35.222932,126.881636
2,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),1,2,unknown,품종견,광주 광산,0,0,0,0,35.222932,126.881636
3,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,7,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,갈색,믹스견,경기 수원,0,2,0,0,37.290757,127.016914
4,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,10,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),1,2,검정색,품종견,경기 성남,0,2,0,1,37.367098,127.127694


# 데이터 전처리

In [ ]:
# 공고기간이 nan인 경우 날림
df.drop(index=df[df['공고번호'].isna()].index,inplace=True)
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421761 entries, 0 to 421760
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   공고번호       421761 non-null  object
 1   축종         421761 non-null  object
 2   품종         420565 non-null  object
 3   털색         421757 non-null  object
 4   성별         421761 non-null  object
 5   중성화여부      421761 non-null  object
 6   특징         421716 non-null  object
 7   접수일시       421761 non-null  object
 8   구조사유       76611 non-null   object
 9   발생장소       421761 non-null  object
 10  공고기간       421761 non-null  object
 11  관할보호센터명    421760 non-null  object
 12  보호장소       421760 non-null  object
 13  관할기관       421761 non-null  object
 14  상태         421761 non-null  object
 15  성격         421761 non-null  int64 
 16  건강         421761 non-null  int64 
 17  나이         421761 non-null  int64 
 18  gender     421761 non-null  int64 
 19  neutering  421761 non-null  int64 
 20  털색2 

In [ ]:
# '공고기간'(str) -> 수치형 데이터의 기간/날짜로 변환
def datediff(d):
    dates = d.split('~')
    date1 = dates[0].strip()
    date2 = dates[1].strip()
    
    y = max(int(date1[:4]),int(date2[:4]))
    date1 = str(y)+date1[4:]
    date2 = str(y)+date2[4:]
    

    date1 = dt.strptime(date1, "%Y-%m-%d")
    date2 = dt.strptime(date2, "%Y-%m-%d")
    
    
    date_diff = date2 - date1
    return abs(date_diff.days)
    
df['공고기간'] = df['공고기간'].map(datediff)
# '접수일시'(str) -> 데이트 타입 변환
df['접수일시'] = pd.to_datetime(df['접수일시'], infer_datetime_format=True) 

In [ ]:
# 축종 
# 개 / 고양이.. 
# 기타 날릴 것
delete_lst = []
for idx, sp in enumerate(df['축종']):
  if sp == '기타축종':
    delete_lst.append(idx)
df.drop(delete_lst,inplace=True)  

# 품종
# 개 - 기타
df['품종'].unique()
df[df['축종']== '개']['품종'].unique()
# 고양이 - 한국고양이
# df['품종'].unique()
# df[df['축종']== '고양이']['품종'].unique()


# 성별
# 암컷 0
# 수컷 1
# 미상 2 
arr = []
for idx,s in enumerate(df['성별']):
  if s == '암컷':
    arr.append(0)
  elif s == '수컷':
    arr.append(1)
  else:
    arr.append(2)
df['gender'] = arr
gender = ['암컷','수컷','미상']
# 중성화여부
# 예 0
# 아니면 1
# 미상 2
arr = []
for idx,s in enumerate(df['중성화여부']):
  if s == '예':
    arr.append(0)
  elif s == '아니오':
    arr.append(1)
  else:
    arr.append(2)
df['neutering'] = arr
neutering = ['예','아니오','미상']

# 털색
# 유사 값 처리 ↓


In [ ]:
# # 색깔 단어 분리
# # Okt, Hannanum, Kkma,Komoran, Mecab
# tokenizer = Okt()
# color_dict = {}

# for color in df['털색'].unique():
#   if type(color) == str:
#     print(color,' -> ',end=' ')
#     temp = tokenizer.nouns(color)
#     for c in temp:
#       if c not in color_dict.keys():
#         color_dict[c] = 1
#       else:
#         color_dict[c] = color_dict[c] + 1
#     print(temp)
#   else:
#     print(color)
# # for color in df['털색'].unique():
# #   print(color)

In [ ]:
# 색 분류
# color_dict = sorted(color_dict.items(),key=lambda c : c[1],reverse=True)
# color_dict
color = {
    '흰색':['흰색','크림','햐양','흰','흰샥','백색','화이트','하양','하얀색','희색','벡','벡색','벡섹','백구','힌색','횐색','휜색','횐','흰섹','힁색','힁','흰샛','화이','배색','진도견'],
    '검정색':['검정','검덩','검졍','검정색','검은색','흑','블랙','흑색','검색','흑갈색','다크','검은빛','검황','검회','검점','까만색','검혼','불랙','검절','겅','검종'],
    '갈색':['갈색','걀색','고둥','갈','브라운','연가','밤색','고동색','초코','고동','밤색','초콜릿','쵸코','흙','흙색','갈섹','암갈색','진밤','커피','초코렛','다갈색','칡색','걸색','초콜렛','동색','강색','칡','갈샥','회걀색','간색'],
    '회색':['회색','진회색','그레이','재색','실버','연회색','쥐색','쥐','잿빛','진회','회색빛','회','회갈색','은색','은회색','은','회검','은빛','길색','연회','잿색','철색','러시안블루','차콜'],
    '황색':['황색','환색','확색','황','황토색','황토','담황색','황백색','보리','누런빛','누런색','누런','누렁','누렁색','진도','황검','회황색','항색','황회','황섹','황새'],
    '노랑색':['노랑','노런','노랑색','노','노란색','연노랑','노라','옐로','노란빛','레몬','연노','진노','옐로우','노검','노혼','노란섹','노랭','누렇색','치즈'],
    '주황색':['주황색','주황','누렁이','오렌지색','오렌지','주','연주황색','연주황','호랑','붉은색','레드','적색','빨간색','홍','붉은빛','홍색','빨','빨강','정색','레즈'],
    '금색':['금색','골드','골든','순금','금','금은','황금','금빛'],
    '살구색':['살구색','살색','살구','분홍','분홍색','핑크','핑크색','연분홍'],
    '고등어':['고등어','고등'],
    '베이지':['베이지','베이지색','베이직'],
    '아이보리':['아이보리','크림'],
    '삼색':['삼색'],
    '턱시도':['턱','시도'],
    '카오스':['카오스']
}
# color_dict

In [ ]:
tokenizer = Okt()
color_feat = []
for cr in df['털색']:
  color_sum = ''
  if isinstance(cr,str):
    temp = tokenizer.nouns(cr)
    if len(temp) == 0:
      color_sum += 'unknown'
    for c in temp:
      for key in color.keys():
        if c in color[key]:
          color_sum += key + ' '
    if color_sum == '':
      color_sum += 'unknown'
  else:
    color_sum += 'unknown'
  color_feat.append(color_sum.strip())  
df['털색2'] = color_feat

In [ ]:
 len(color_feat)

421761

In [ ]:
# okt = Okt() 

# word_dict = {}
# for dog_breed in df[df['축종']=='개']['품종']:
#   if type(dog_breed) == str:
#     print(dog_breed,' -> ',end=' ')
#     temp = okt.nouns(dog_breed)
#     for db in temp:
#       if db not in word_dict.keys():
#         word_dict[db] = 1
#       else:
#         word_dict[db] = word_dict[db] + 1
#     print(temp)
#   else:
#     print(dog_breed)

# word_dict = sorted(word_dict.items(),key=lambda db : db[1], reverse=True)


# c_word_dict = {}
# for cat_breed in df[df['축종']=='고양이']['품종']:
#   if type(cat_breed) == str:
#     print(cat_breed,' -> ',end=' ')
#     temp2 = okt.nouns(cat_breed)
#     for cb in temp2:
#       if cb not in c_word_dict.keys():
#         c_word_dict[cb] = 1
#       else:
#         c_word_dict[cb] = c_word_dict[cb] + 1
#     print(temp2)
#   else:
#     print(cat_breed)
    
# c_word_dict = sorted(c_word_dict.items(),key=lambda cb : cb[1], reverse=True)

In [ ]:
okt.nouns('mix')

[]

In [ ]:
breed = {
    '믹스견' : ['믹스견','믹스','혼합','기타','잡종','잡종견','들개','재래종','혼종','혼혈','모름'],
    '한국 고양이' : ['고양이','한국','코숏','기타','고등어','치즈','혼종',
                '턱시도','시도','턱','잡종','삼색','태비','젖소','들고양이','잡종',
                '코리안','야생고양이','길','모름','코쇽','무늬','콧숏','미상',
                '믹스견','혼','알수없음','추정', '한국 고양이','야생', '쇼컷']
}
okt = Okt() 

def dbreed(data): 
    if data == '푸들':
        return '품종견'
    if isinstance(data, str):
        temp = okt.nouns(data)
        status = 0
        if len(temp)==0:
            return '믹스견'
        
        for i in temp:        
            if i in breed['믹스견']:
                status =1

        if status == 0:
            return '품종견'
        else : # bread['믹스견']에 들어갔다가 나왔.
            if len(temp) >= 2:  # 단어 개수 2개 이상
                return '품종 믹스견'
            else:               # 단어 개수 1개
              if ('혼혈' in temp) or ('혼종' in temp) or ('혼합' in temp)  :
                return '품종 믹스견'
              else:  
                return '믹스견' 
    else: return '믹스견'
        
def cbreed(data):  
    if isinstance(data, str): # 문자열 일 경우
        temp = okt.nouns(data)
        status = 0
        
        if len(temp)==0:
            return '한국 고양이'
        
        for i in temp:        
            if i in breed['한국 고양이']:
                status =1

        if status == 0:
          if '믹스' in temp:
            return '품종 믹스묘'
          else : 
            return '품종묘'
        else :
          if '믹스' in temp:
            return '품종 믹스묘'
          else :
            return '한국 고양이'        
    else: return '한국 고양이'  # 문자열 x
    

In [ ]:
breed_lst=[]
for i in df[['축종','품종']].values:
    if i[0] == '개':
        breed_lst.append(dbreed(i[1]))
    else:
        breed_lst.append(cbreed(i[1]))
        
df['품종2'] = breed_lst

In [ ]:
# 공고 번호 -> 지역 칼럼 추가
locations = []
for i in df['공고번호']:
  if isinstance(i,str):
    temp = i.split('-')
    locations.append(temp[0]+' '+temp[1])
  else:
    locations.append('unknown')
df['지역'] = locations
df.head()


,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
0,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견,광주 남구
1,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견,광주 남구
2,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견,광주 광산
3,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,7,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견,경기 수원
4,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,10,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),0,2,0,1,2,검정색,품종견,경기 성남


In [ ]:
# 공고번호 빈거 하나 날리기
df.drop(index=df[df['공고번호'].isna()].index ,axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 0 to 421760
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   공고번호       421761 non-null  object        
 1   축종         421761 non-null  object        
 2   품종         420565 non-null  object        
 3   털색         421757 non-null  object        
 4   성별         421761 non-null  object        
 5   중성화여부      421761 non-null  object        
 6   특징         421716 non-null  object        
 7   접수일시       421761 non-null  datetime64[ns]
 8   구조사유       76611 non-null   object        
 9   발생장소       421761 non-null  object        
 10  공고기간       421761 non-null  int64         
 11  관할보호센터명    421760 non-null  object        
 12  보호장소       421760 non-null  object        
 13  관할기관       421761 non-null  object        
 14  상태         421761 non-null  object        
 15  성격         421761 non-null  int64         
 16  건강         421761 no

In [ ]:
# 잘못 크롤링 되어 비어버린 상태 값 날리기
df.drop(index=df[df['상태'].isna()].index, axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 0 to 421760
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   공고번호       421761 non-null  object        
 1   축종         421761 non-null  object        
 2   품종         420565 non-null  object        
 3   털색         421757 non-null  object        
 4   성별         421761 non-null  object        
 5   중성화여부      421761 non-null  object        
 6   특징         421716 non-null  object        
 7   접수일시       421761 non-null  datetime64[ns]
 8   구조사유       76611 non-null   object        
 9   발생장소       421761 non-null  object        
 10  공고기간       421761 non-null  int64         
 11  관할보호센터명    421760 non-null  object        
 12  보호장소       421760 non-null  object        
 13  관할기관       421761 non-null  object        
 14  상태         421761 non-null  object        
 15  성격         421761 non-null  int64         
 16  건강         421761 no

In [ ]:
# df.to_csv('/content/drive/MyDrive/TwoPs/AniHelp_feat3.csv',encoding='utf-8')

## '특징' 데이터 처리


In [ ]:
# # 단어들 (명사, 동사, 형용사) 빈도 보기
# okt = Okt()
# feat_arr = [] # 
# word_dic = {}
# for word in df['특징']:
#   if isinstance(word, str):
#     temp = okt.pos(word, norm=True, stem =True)
#     feat = []
#     for w in temp:
#       if w[1] in ['Noun','Verb','Adjective']:
#         feat.append(w[0])
#         if not (w[0] in word_dic):
#           word_dic[w[0]] = 0
#         word_dic[w[0]] += 1
#     feat_arr.append(feat)


In [ ]:
# # 빈도가 높은 순 정렬
# word_dic = sorted(word_dic.items(), key= lambda item: item[1],reverse=True)
# word_dic

NameError: ignored

In [ ]:
# 특징 feature를 특정 카테고리로 분류하기 위함
character_ct = { # -1  +2
    1 : ['경계','계심','겁','사나','소심하다','의심','짖다','공격','사납다','공격성',
         '으르렁','무섭다','불안','신경','힘없다','겁쟁이','산만하다','불안하다','낯','약간'],
    2 : ['좋아하다','붙임성','사람','순하다','온순하다','얌전하다','활발하다','따르다','좋다','착하다',
         '애교','영리하다','밝다','친화력','활달함','명랑하다','친해지다','차분하다','조용하다',
         '활달함','에너지','점잖다','순진하다','따름','친근하다', '착하다']
}
health_ct = {    # -1  --2
    1 : ['빨갛다','미흡','불량','여위다','결막염','혼탁','기력','허약','쇄약','눈곱','오염','오물',
         '저하','설사','탈수','마름','영양실조','허약하다','눈꼽심함','힘없다','구취','노란눈꼽','악취',
         '학대','쇠약하다','눈꼽기','눈꼽많음','저체중','부실','말랐슴','침','흘리다','저하','설사',
         '탈수','진드기','접종','백신','예방','장염','구내염','콧물심함','경미하다'],
    2 : ['상태','부다','심하다','교상','불능','결막염','혼탁','빠지다','털','교통사고','심각','상처',
         '골절','피부','질병','마비','염증','양호','피부병','척추','뒷다리','부정교합','이염', '심장',
         '피부질환','탈장','회충','절단','골절','질환','사상충','피부','백내장','출혈','안좋다','부상',
         '사고','다리부상','앞다리','탈모','치료','내장','이상','다치다','뒷발','증상','손상','잃다','수술',
         '나쁘다','소실','종양','질병','구더기','눈병','구강','괴사','전염','장애','기생충','돌출',
         '광견병','각막','복부','기관지염','절단','아프다','불편','불량하다','흉터','파열','궤양','바이러스',
         '실명','구충','하반신마비','뼈','혈변','기형','감염증','찢어지다','폐렴','귓병','슬개골탈구',
         '고름','거동','모낭충','벗겨지다','척추골절','아래턱','전염병','안염','혼수상태','교통','코마',
         '발작','통증','회충','짖무름','허피스바이러스','쇼크','절뚝거림','회충증','황달','응급치료','패혈증',
         '부실','내출혈','탈모증','진물','낙상','각막염','녹내장','종기','중상']
}
age_ct = {
    1: ['개월','월령','개체','관리','어리다','새끼','생후','강아지','아가','젖먹이','나이','애기','새끼고양이','이유식','분유','꼬물이','젖','갓난']
}

In [ ]:
# 특징 feature로 부터 새로운 데이터 파생
character = np.zeros(df.shape[0],dtype=np.int16)
health = np.zeros(df.shape[0],dtype=np.int16)
age = np.zeros(df.shape[0],dtype=np.int16)

okt = Okt()

for idx,word in enumerate(df['특징']):
  if isinstance(word, str):
    temp = okt.pos(word, norm=True, stem =True)
  
    for w in temp:
      if w[1] in ['Noun','Verb','Adjective']:
        for c in character_ct: # - 1, + 2
          if w[0] in character_ct[c]:
            if (c== 1) and ('없다' in temp):
              character[idx] = 2
            character[idx] = c
        for h in health_ct:
          if w[0] in health_ct[h]:
            health[idx] = h
        for a in age_ct:
          if w[0] in age_ct[a]:
            age[idx] = a  
        

d = {
    '성격':character,
    '건강':health,
    '나이':age
}
df2 = pd.DataFrame(d)


In [ ]:
df.drop(['건강','성격','나이'],axis=1,inplace=True)

# 파생된 특징 dataframe 합치기
df = pd.concat([df,df2], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 0 to 421760
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   공고번호       421761 non-null  object        
 1   축종         421761 non-null  object        
 2   품종         420565 non-null  object        
 3   털색         421757 non-null  object        
 4   성별         421761 non-null  object        
 5   중성화여부      421761 non-null  object        
 6   특징         421716 non-null  object        
 7   접수일시       421761 non-null  datetime64[ns]
 8   구조사유       76611 non-null   object        
 9   발생장소       421761 non-null  object        
 10  공고기간       421761 non-null  int64         
 11  관할보호센터명    421760 non-null  object        
 12  보호장소       421760 non-null  object        
 13  관할기관       421761 non-null  object        
 14  상태         421761 non-null  object        
 15  gender     421761 non-null  int64         
 16  neutering  421761 no

In [ ]:
# df.drop(index=0,inplace=True)
# df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,gender,neutering,털색2,품종2,지역,성격,건강,나이
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,10.0,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),1.0,2.0,흰색,믹스견,광주 남구,0.0,0.0,0.0
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,10.0,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0.0,2.0,흰색,믹스견,광주 남구,0.0,0.0,0.0
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,10.0,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),1.0,2.0,unknown,품종견,광주 광산,0.0,2.0,0.0
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,7.0,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0.0,2.0,갈색,믹스견,경기 수원,0.0,2.0,0.0
5,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,10.0,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),1.0,2.0,검정색,품종견,경기 성남,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 0 to 421760
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   공고번호       421761 non-null  object        
 1   축종         421761 non-null  object        
 2   품종         420565 non-null  object        
 3   털색         421757 non-null  object        
 4   성별         421761 non-null  object        
 5   중성화여부      421761 non-null  object        
 6   특징         421716 non-null  object        
 7   접수일시       421761 non-null  datetime64[ns]
 8   구조사유       76611 non-null   object        
 9   발생장소       421761 non-null  object        
 10  공고기간       421761 non-null  int64         
 11  관할보호센터명    421760 non-null  object        
 12  보호장소       421760 non-null  object        
 13  관할기관       421761 non-null  object        
 14  상태         421761 non-null  object        
 15  gender     421761 non-null  int64         
 16  neutering  421761 no

In [ ]:
df[df['공고기간']>300]

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,gender,neutering,털색2,품종2,지역,성격,건강,나이
2832,경남-창녕-2018-00119,개,믹스견,흰색,암컷,아니오,없음,2018-01-11,NaN,유어면 진창리,329,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28,경상남도 창녕군,종료(안락사),0,2,흰색,믹스견,경남 창녕,0,0,0
3048,경남-창녕-2018-00120,개,믹스견,검정,수컷,아니오,없음,2018-01-12,NaN,대합면 십이리,329,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28,경상남도 창녕군,종료(자연사),1,2,검정색,믹스견,경남 창녕,0,0,0
3049,경남-창녕-2018-00121,개,믹스견,흰색,암컷,아니오,없음,2018-01-12,NaN,대합면 십이리,329,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28,경상남도 창녕군,종료(안락사),0,2,흰색,믹스견,경남 창녕,0,0,0
3050,경남-창녕-2018-00122,개,믹스견,검정,암컷,아니오,없음,2018-01-12,NaN,대합면 십이리,329,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28,경상남도 창녕군,종료(안락사),0,2,검정색,믹스견,경남 창녕,0,0,0
3051,경남-창녕-2018-00123,개,믹스견,검정,수컷,아니오,없음,2018-01-12,NaN,대합면 십이리,329,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28,경상남도 창녕군,종료(안락사),1,2,검정색,믹스견,경남 창녕,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369174,경남-창원2-2020-00515,개,믹스견,연갈+아이보리+검정,암컷,아니오,"순함, 등부분 검정털, 자견 5마리와 함께 입소",2020-12-31,NaN,지하련 생가 (합포북 13길 41-1) 옆 폐가,354,마산유기동물보호소,경상남도 창원시 마산합포구 진북면 지산2길 139-112 226-19,경상남도 창원시 마산합포회원구,종료(입양),0,2,아이보리 검정색,믹스견,경남 창원2,2,2,0
369177,경기-광명-2020-00345,개,닥스훈트,흑갈,암컷,미상,"순함,노란하네스",2021-01-01,NaN,소하동 1373 부근,352,한국야생동물보호협회,경기도 안산시 상록구 청곡길 50 (부곡동) 한국야생동물보호협회,경기도 광명시,종료(반환),0,2,unknown,품종견,경기 광명,2,0,0
369178,경기-광명-2020-00346,개,믹스견,백갈,수컷,아니오,"8개월추정,순함,빨간목줄",2021-01-01,NaN,광명시청에서 인수,352,한국야생동물보호협회,경기도 안산시 상록구 청곡길 50 (부곡동) 한국야생동물보호협회,경기도 광명시,보호중,1,2,unknown,믹스견,경기 광명,2,1,1
369181,경기-광명-2020-00344,개,믹스견,백갈,암컷,아니오,"2개월추정,분홍색옷,겁많음,입질살짝있음",2021-01-01,NaN,철산동 467-81 부근,352,한국야생동물보호협회,경기도 안산시 상록구 청곡길 50 (부곡동) 한국야생동물보호협회,경기도 광명시,종료(입양),0,2,unknown,믹스견,경기 광명,1,0,1


In [ ]:
df.to_csv('/content/drive/MyDrive/TwoPs/AniHelp_feat4.csv',encoding='utf-8')

### 특징을 잠재 의미를 분석하는 토픽 모델에 적용
##### 특징들을 토픽별로 잘 모으는 것 같지 않음

In [ ]:
# LDA ( 잠재 디클레리 분석) 을 위한 전처리
# 단어들을 토큰화하여 LDA 모델 입력 준비
okt = Okt()
st_arr = [] # 

for word in df['특징']:
  if isinstance(word, str):
    temp = okt.pos(word, norm=True, stem =True)
    sentence = []
    for w in temp:
      if w[1] in ['Noun','Verb','Adjective']:
        sentence.append(w[0])
    st_arr.append(sentence)


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(st_arr) # (word_id, word_frequency) 의 형태를 만들기 위해 단어 사전 생성
corpus = [dictionary.doc2bow(text) for text in st_arr] # 각 문장당 (word_id, word_frequency) 형태로 인코딩
print(corpus[1]) # 수행된 결과에서 두번째 문장 출력. 첫번째 문서의 인덱스는 0

[(1, 1)]


In [ ]:
# LDA 모델 학습
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) # passes는 알고리즘의 동작 횟수, 
topics2 = ldamodel.print_topics(num_words=30) # num_words=4로 총 4개의 단어만 출력 
for topic in topics2:
    print(topic) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트" + 0.001*"낯선" + 0.001*"관찰" + 0.001*"사이" + 0.001*"결막" + 0.001*"파랑" + 0.001*"들어가다" + 0.001*"파출소" + 0.001*"불안" + 0.001*"이주" + 0.001*"집앞"')
(1, '0.134*"심하다" + 0.102*"입소" + 0.067*"피부병" + 0.058*"유기" + 0.051*"박스" + 0.036*"접종" + 0.033*"필요하다" + 0.029*"번" + 0.023*"치료" + 0.022*"버려지다" + 0.021*"예쁘다" + 0.021*"담기다" + 0.020*"친" + 0.017*"필요" + 0.017*"적" + 0.013*"병원" + 0.013*"묘" + 0.012*"사납다" + 0.009*"구충" + 0.008*"악질" + 0.008*"넣다" + 0.006*"힘" + 0.006*"공격성" + 0.006*"달반" + 0.006*"종이" + 0.005*"옆" + 0.005*"바람" + 0.005*"고자" + 0.005*"응원" + 0.005*"케이"')
(2, '0.100*"교통사고" + 0.058*"골절" + 0.052*"뒷다리" + 0.040*"후지" + 0.037*"및" + 0.036*"추정" + 0.026*"등록" + 0.024*"예후" + 0.022*"불량" + 0.022*"내장" + 0.021*"염증" + 0.019*"출혈" + 0.019*"우측" + 0.017*"이상" + 0.016*"동시"

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# 주제가 10개
topics2 = ldamodel.print_topics(num_words=100) # num_words=30로 총 30개의 단어만 출력 
for topic in topics2:
    print(topic) 

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트" + 0.001*"낯선" + 0.001*"관찰" + 0.001*"사이" + 0.001*"결막" + 0.001*"파랑" + 0.001*"들어가다" + 0.001*"파출소" + 0.001*"불안" + 0.001*"이주" + 0.001*"집앞" + 0.001*"성대수술" + 0.001*"생리" + 0.001*"몸매" + 0.000*"검색" + 0.000*"기르다" + 0.000*"안과" + 0.000*"쇄약" + 0.000*"시장" + 0.000*"파목줄" + 0.000*"노목줄" + 0.000*"순딩" + 0.000*"앵무새" + 0.000*"후분" + 0.000*"청평" + 0.000*"무상처" + 0.000*"원피스" + 0.000*"홍일점" + 0.000*"요추" + 0.000*"녹목줄" + 0.000*"올인원" + 0.000*"카키목줄" + 0.000*"남양" + 0.000*"초목" + 0.000*"김화" + 0.000*"군복" + 0.000*"둘" + 0.000*"뱀" + 0.000*"개성" + 0.000*"남움" + 0.000*"다시다" + 0.000*"노색목줄" + 0.000*"변산면" + 0.000*"탄탄하다" + 0.000*"컨" + 0.000*"목즐" + 0.000*"빨파목줄" + 0.000*"봉인" + 0.000*"주홍" + 0.000*"잘생겻음" + 0.000*"색줄" + 0.000*"좌천" + 0.000*"우수관" + 0.000*"관릴" + 0.000*"등외" + 0.000

In [ ]:
# 주제가 20개
topics = ldamodel.print_topics(num_words=20) # num_words=4로 총 4개의 단어만 출력 
for topic in topics:
    print(topic) 

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트"')
(1, '0.134*"심하다" + 0.102*"입소" + 0.067*"피부병" + 0.058*"유기" + 0.051*"박스" + 0.036*"접종" + 0.033*"필요하다" + 0.029*"번" + 0.023*"치료" + 0.022*"버려지다" + 0.021*"예쁘다" + 0.021*"담기다" + 0.020*"친" + 0.017*"필요" + 0.017*"적" + 0.013*"병원" + 0.013*"묘" + 0.012*"사납다" + 0.009*"구충" + 0.008*"악질"')
(2, '0.100*"교통사고" + 0.058*"골절" + 0.052*"뒷다리" + 0.040*"후지" + 0.037*"및" + 0.036*"추정" + 0.026*"등록" + 0.024*"예후" + 0.022*"불량" + 0.022*"내장" + 0.021*"염증" + 0.019*"출혈" + 0.019*"우측" + 0.017*"이상" + 0.016*"동시" + 0.016*"연제구" + 0.015*"안구" + 0.014*"마비" + 0.014*"낑낑대다" + 0.013*"좌측"')
(3, '0.349*"없다" + 0.107*"기력" + 0.090*"칩" + 0.062*"설사" + 0.054*"영양" + 0.047*"생기다" + 0.032*"체온" + 0.029*"애교" + 0.028*"주일" + 0.023*"포획틀" + 0.020*"금정" + 0.010*"주황색" + 0.009*"라인" + 0.008*"특징" + 0.

In [ ]:
# LDA 시각화 툴 설치
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 3.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7MB 182kB/s 
     |████████████████████████████████| 9.9MB 36.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=1ed97063a75b07a6800e59ba5b634b5b4af461114c31f9df0c2a1d7491c269e1
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


ModuleNotFoundError: ignored

In [ ]:
# 크롬 개발자 도구 console 창에 입력하면 90분 뒤 아무 조작없음이 안뜨도록 계속 클릭을 해줌
# function ClickConnect() {
#     var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
#     buttons.forEach(function(btn) { 
#         btn.click(); 
#     }); 
#     console.log("1분마다 자동 재연결"); 
#     document.querySelector("colab-toolbar-button#connect").click(); 
# } 
# setInterval(ClickConnect,1000*60);

# 데이터 분석

##### 데이터 불러오기

In [ ]:
# 정희 데이터 불러오는 부분
jeonghee_path = '/content/drive/MyDrive/TwoPs/AniHelp_feat3.csv'
df = pd.read_csv(jeonghee_path, index_col=0)
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견,광주 남구
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견,광주 남구
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견,광주 광산
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견,경기 수원
5,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,2018-01-01 ~ 2018-01-11,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),0,2,0,1,2,검정색,품종견,경기 성남


#### 1차 가설

1. 축종 : 개/고양이 3:1
  - 개가 높나 고양이가 높나(입양률 비교)

1. 품종 : 믹스/품종믹스/품종/모름
  - 개는 소형견이 
  - 고양이는 품종묘가 더 높을것
1. 성별 : 암/수/모름
  - 품종에 한해서? 암컷높을 것
1. 중성화여부: y/n/모름
  - 크게 영향 안미칠 것 같
1. 털색 : (주관식)
  - 흰색이 높을 것 같고, 그 외에는 특이한 무늬가 높을 것(달마시안)
1. 연도별 : 입양여부
  - 연도별 in/out되는 축종 비교 해보면 고양이의 in이 많아질 것. 
  - 소가족 비율, 자취생 비율, 
  - 사회구조 변화 등 집에서 활동률 높아짐. (2020-21)
  - 평균 공고 기간의 변화
1. 지역 
  - 지역별 income되는 숫자
  - 지역별 outcome되는 숫자
  - 지역별 공고 기간
1. 특징 : 성격/건강/나이
  - 성격: 사람 친화적인 아이들이 잘 나갈 것
  - 건강: 큰 질병이 없는 아이들
  - 나이: (고양이)어릴 수록 잘 나갈 거라 생각됨, 
1. 공고기간 
  - 7.10.12.14 정도로 값이 있던데, 길수록 (공고기간 지나서도 보호중인 아이들도 있음) 입양률 높아질 것 >> 
 